In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from zipfile import ZipFile
import pandas as pd
import torch
import random
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm, trange

import sys
sys.path.append('..')

from src.DMHP.HP import PointProcessStorage, DirichletMixtureModel, EM_clustering, tune_basis_fn
from src.DMHP.metrics import consistency, purity
from src.Cohortney.data_utils import load_data

In [11]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K2_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 2

0.24024024024024024
D = 6
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 93.6080
1it [01:12, 72.86s/it]
NLL / N: 89.5990
2it [01:18, 52.62s/it]
NLL / N: 86.2993
3it [01:25, 38.86s/it]
NLL / N: 85.4844
4it [01:32, 29.53s/it]
NLL / N: 85.9212
5it [01:42, 23.45s/it]
NLL / N: 85.5303
6it [01:52, 19.49s/it]
NLL / N: 102.5173
7it [02:03, 17.10s/it]
NLL / N: 101.5094
8it [02:15, 15.43s/it]
NLL / N: 97.6430
9it [02:26, 14.26s/it]
NLL / N: 95.2362
10it [02:38, 15.83s/it]
preds: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0

1it [01:11, 71.97s/it]
NLL / N: 94.0338
2it [01:17, 51.95s/it]
NLL / N: 92.5571
3it [01:23, 38.35s/it]
NLL / N: 88.0104
4it [01:31, 29.19s/it]
NLL / N: 101.2081
5it [01:40, 23.12s/it]
NLL / N: 100.8583
6it [01:50, 19.25s/it]
NLL / N: 97.1273
7it [02:02, 16.96s/it]
NLL / N: 95.3795
8it [02:13, 15.33s/it]
NLL / N: 95.0933
9it [02:25, 14.31s/it]
NLL / N: 94.9586
10it [02:37, 15.75s/it]
preds: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,

4it [01:36, 30.83s/it]
NLL / N: 99.3593
5it [01:45, 24.33s/it]
NLL / N: 100.9362
6it [01:55, 20.08s/it]
NLL / N: 96.8137
7it [02:07, 17.61s/it]
NLL / N: 95.0667
8it [02:19, 15.75s/it]
NLL / N: 94.9556
9it [02:30, 14.51s/it]
NLL / N: 94.9541
10it [02:42, 16.21s/it]
preds: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        

In [12]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K3_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 4

0.46546546546546547
D = 15
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 112.6410
1it [05:34, 334.85s/it]
NLL / N: 113.3026
2it [05:43, 236.96s/it]
NLL / N: 112.3344
3it [05:55, 169.57s/it]
NLL / N: 114.7810
4it [06:09, 122.82s/it]
NLL / N: 118.4918
5it [06:29, 91.97s/it] 
NLL / N: 118.1865
6it [06:49, 70.55s/it]
NLL / N: 119.0611
7it [07:10, 55.64s/it]
NLL / N: 117.7443
8it [07:30, 44.94s/it]
NLL / N: 116.1772
9it [07:51, 37.79s/it]
NLL / N: 115.1400
10it [08:12, 49.22s/it]
preds: tensor([1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1.,
        1., 2., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
        1., 1.,

0it [00:00, ?it/s]
NLL / N: 111.4573
1it [05:27, 327.33s/it]
NLL / N: 115.7589
2it [05:35, 231.60s/it]
NLL / N: 117.1444
3it [05:45, 165.22s/it]
NLL / N: 130.1947
4it [05:58, 119.38s/it]
NLL / N: 128.9801
5it [06:12, 87.87s/it] 
NLL / N: 124.3212
6it [06:29, 66.53s/it]
NLL / N: 118.1560
7it [06:48, 52.28s/it]
NLL / N: 114.7578
8it [07:06, 42.13s/it]
NLL / N: 114.1698
9it [07:25, 35.01s/it]
NLL / N: 114.0105
10it [07:43, 46.35s/it]
preds: tensor([3., 1., 3., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 3., 1., 3., 3.,
        1., 1., 1., 3., 1., 3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3.,
        1., 3., 1., 1., 3., 1., 3., 3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 1.,
        1., 1., 3., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 1., 3., 1., 3., 3.,
        3., 1., 3., 3., 3., 3., 1., 3., 3., 3., 1., 1., 3., 1., 3., 3., 3., 3.,
        1., 3., 3., 3., 3., 1., 1., 1., 3., 3., 1., 1., 1., 3., 1., 1., 1., 3.,
        1., 1., 1., 1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.

0it [00:00, ?it/s]
NLL / N: 125.2248
1it [05:24, 324.74s/it]
NLL / N: 122.3980
2it [05:35, 230.55s/it]
NLL / N: 120.3969
3it [05:48, 165.39s/it]
NLL / N: 121.3220
4it [06:05, 120.64s/it]
NLL / N: 131.8497
5it [06:19, 88.82s/it] 
NLL / N: 146.1759
6it [06:40, 68.43s/it]
NLL / N: 146.9886
7it [06:59, 53.61s/it]
NLL / N: 120.6698
8it [07:18, 43.10s/it]
NLL / N: 119.3149
9it [07:36, 35.83s/it]
NLL / N: 117.6796
10it [07:55, 47.55s/it]
preds: tensor([0., 2., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
        0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 2.,
        0., 2., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 2., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.

In [13]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K4_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 4

/mnt/Cohortney/src/mds20_cohortney-main/src/DMHP/HP.py:26: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result = lambda w0: integrate.quad(upper_bound, w0, np.inf)
0.7207207207207207
D = 20
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 108.5944
1it [09:56, 596.09s/it]
NLL / N: 108.8296
2it [10:09, 421.40s/it]
NLL / N: 110.7738
3it [10:23, 299.19s/it]
NLL / N: 112.8909
4it [10:40, 214.53s/it]
NLL / N: 112.1896
5it [11:00, 156.07s/it]
NLL / N: 112.6377
6it [11:24, 116.42s/it]
NLL / N: 112.4251
7it [11:53, 90.23s/it] 
NLL / N: 111.9132
8it [12:23, 72.03s/it]
NLL / N: 112.1460
9it [12:52, 59.18s/it]
NLL / N: 112.3112
10it [13:21, 80.10s/it]
preds: tensor([1., 1., 3., 1., 1., 1., 1., 1., 3., 3., 3., 1., 3., 1., 1., 3., 1., 3.,
        3., 1., 1., 1., 1., 1., 1., 3., 3., 3., 3., 1., 1., 1., 1., 1., 1., 3.,
        3., 1., 3., 3., 3., 1., 3., 3., 3., 3.,

6it [11:40, 119.18s/it]
NLL / N: 113.4491
7it [12:07, 91.57s/it] 
NLL / N: 114.3378
8it [12:36, 72.75s/it]
NLL / N: 114.7295
9it [13:01, 58.64s/it]
NLL / N: 114.7570
10it [13:27, 80.71s/it]
preds: tensor([3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 2., 3.,
        3., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 0., 3., 0., 3., 3., 3., 2.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 0., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 2., 3., 2., 3., 3.,
        3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3.,
        2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3.,
   

0it [00:00, ?it/s]
NLL / N: 104.6392
1it [09:56, 596.55s/it]
NLL / N: 104.0286
2it [10:09, 421.59s/it]
NLL / N: 104.8929
3it [10:25, 299.80s/it]
NLL / N: 106.9375
4it [10:44, 215.53s/it]
NLL / N: 111.3929
5it [11:07, 157.74s/it]
NLL / N: 114.3728
6it [11:33, 118.41s/it]
NLL / N: 115.5595
7it [12:07, 92.93s/it] 
NLL / N: 115.2939
8it [12:36, 73.80s/it]
NLL / N: 114.8200
9it [13:05, 60.24s/it]
NLL / N: 114.7178
10it [13:34, 81.42s/it]
preds: tensor([1., 1., 2., 2., 2., 1., 1., 2., 2., 2., 2., 1., 2., 1., 1., 2., 1., 2.,
        2., 2., 1., 2., 1., 1., 2., 2., 2., 2., 2., 2., 1., 1., 2., 1., 2., 2.,
        2., 2., 2., 2., 2., 1., 2., 2., 2., 2., 1., 2., 2., 1., 2., 2., 2., 1.,
        2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 2., 1., 2., 3., 2., 2., 2., 1.,
        2., 2., 2., 2., 2., 2., 2., 1., 2., 2., 1., 2., 2., 2., 2., 1., 2., 1.,
        2., 2., 1., 2., 2., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1.,
        2., 2., 2., 2., 2., 2., 2., 1., 1., 2., 2., 1., 3., 1., 2., 2., 2., 

In [14]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K5_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 5

0.9009009009009009
D = 29
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 106.0546
1it [20:02, 1202.78s/it]
NLL / N: 106.8007
2it [20:26, 849.19s/it] 
NLL / N: 109.6161
3it [20:58, 603.78s/it]
NLL / N: 107.6573
4it [21:21, 429.82s/it]
NLL / N: 114.2009
5it [21:52, 309.95s/it]
NLL / N: 112.9076
6it [22:24, 226.78s/it]
NLL / N: 110.8121
7it [23:02, 169.92s/it]
NLL / N: 109.9726
8it [23:39, 130.12s/it]
NLL / N: 109.2677
9it [24:15, 101.80s/it]
NLL / N: 108.0387
10it [24:49, 148.92s/it]
preds: tensor([1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 2., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
        0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
        1

0it [00:00, ?it/s]
NLL / N: 105.5402
1it [20:16, 1216.48s/it]
NLL / N: 105.7194
2it [20:31, 856.15s/it] 
NLL / N: 106.4447
3it [20:54, 605.96s/it]
NLL / N: 110.7612
4it [21:16, 430.96s/it]
NLL / N: 111.3530
5it [21:43, 309.66s/it]
NLL / N: 113.7427
6it [22:13, 225.95s/it]
NLL / N: 115.8307
7it [22:48, 168.54s/it]
NLL / N: 115.0500
8it [23:23, 128.55s/it]
NLL / N: 113.7145
9it [23:58, 100.41s/it]
NLL / N: 112.7529
10it [24:32, 147.26s/it]
preds: tensor([2., 2., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 2., 4., 4.,
        2., 2., 2., 4., 2., 4., 4., 4., 4., 4., 4., 2., 2., 4., 4., 4., 0., 4.,
        4., 4., 2., 2., 2., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 0., 2.,
        2., 2., 4., 2., 2., 2., 4., 4., 4., 4., 4., 4., 4., 2., 4., 2., 0., 4.,
        4., 2., 4., 4., 2., 4., 2., 4., 4., 4., 4., 4., 4., 4., 2., 4., 4., 2.,
        4., 4., 4., 4., 4., 4., 2., 2., 4., 4., 2., 2., 2., 4., 4., 2., 2., 4.,
        2., 4., 4., 2., 2., 2., 4., 4., 4., 4., 4., 4., 4., 4., 4., 0.,

0it [00:00, ?it/s]
NLL / N: 105.3023
1it [20:03, 1203.15s/it]
NLL / N: 108.7438
2it [20:20, 847.42s/it] 
NLL / N: 111.1051
3it [20:41, 599.42s/it]
NLL / N: 114.2170
4it [21:09, 428.18s/it]
NLL / N: 113.3533
5it [21:41, 309.34s/it]
NLL / N: 114.9190
6it [22:15, 226.72s/it]
NLL / N: 144.9962
7it [22:52, 169.72s/it]
NLL / N: 273.4337
8it [23:30, 130.09s/it]
NLL / N: 406.1927
9it [24:09, 102.92s/it]
NLL / N: 410.5849
10it [24:44, 148.40s/it]
preds: tensor([4., 3., 1., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 1., 4.,
        4., 4., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 3.,
        3., 3., 4., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 3., 4., 4.,
        4., 3., 4., 4., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 1.,

0it [00:00, ?it/s]
NLL / N: 107.2167
1it [20:11, 1211.72s/it]
NLL / N: 102.5360
2it [20:29, 853.59s/it] 
NLL / N: 107.2677
3it [20:50, 603.90s/it]
NLL / N: 108.2763
4it [21:17, 430.54s/it]
NLL / N: 108.0785
5it [21:48, 310.83s/it]
NLL / N: 109.9917
6it [22:22, 227.89s/it]
NLL / N: 115.6457
7it [22:59, 170.66s/it]
NLL / N: 117.8870
8it [23:37, 130.76s/it]
NLL / N: 124.8204
9it [24:16, 103.05s/it]
NLL / N: 158.4805
10it [24:54, 149.49s/it]
preds: tensor([4., 4., 1., 4., 1., 4., 4., 4., 4., 1., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 1., 1.,
        4., 4., 0., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 1., 0.,
        4., 0., 4., 0., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 0., 1., 4.,
        4., 4., 4., 4., 0., 4., 4., 1., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 4., 4., 1., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 1.,

0it [00:00, ?it/s]
NLL / N: 101.8701
1it [20:12, 1212.09s/it]
NLL / N: 107.4593
2it [20:29, 853.81s/it] 
NLL / N: 107.9448
3it [20:54, 605.13s/it]
NLL / N: 107.9502
4it [21:20, 431.21s/it]
NLL / N: 106.7029
5it [21:51, 311.13s/it]
NLL / N: 112.4405
6it [22:26, 228.27s/it]
NLL / N: 111.7537
7it [23:00, 170.19s/it]
NLL / N: 111.1736
8it [23:34, 129.34s/it]
NLL / N: 110.6849
9it [24:10, 101.28s/it]
NLL / N: 110.2037
10it [24:50, 149.03s/it]
preds: tensor([1., 1., 4., 1., 2., 4., 1., 1., 4., 2., 4., 1., 1., 1., 4., 1., 1., 4.,
        1., 1., 1., 4., 1., 4., 1., 1., 1., 4., 1., 1., 1., 1., 4., 4., 4., 1.,
        1., 1., 1., 1., 1., 1., 4., 1., 1., 1., 1., 1., 1., 4., 4., 1., 1., 1.,
        1., 1., 4., 1., 1., 1., 1., 4., 4., 1., 4., 1., 4., 1., 1., 1., 1., 4.,
        4., 1., 1., 4., 1., 4., 1., 1., 4., 4., 1., 1., 1., 1., 1., 1., 4., 1.,
        1., 4., 1., 4., 4., 1., 1., 1., 1., 4., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 4., 1., 4., 4., 1., 1.,

### Sin

In [3]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_sin_K2_C5_1' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 2

1.8318318318318318
D = 12
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 108.4364
1it [06:02, 362.21s/it]
NLL / N: 85.8283
2it [06:13, 256.95s/it]
NLL / N: 90.2656
3it [06:28, 184.42s/it]
NLL / N: 94.8088
4it [06:46, 134.52s/it]
NLL / N: 94.1639
5it [07:08, 100.68s/it]
NLL / N: 94.9524
6it [07:31, 77.43s/it] 
NLL / N: 95.9206
7it [07:57, 61.86s/it]
NLL / N: 96.5344
8it [08:22, 50.94s/it]
NLL / N: 96.9284
9it [08:51, 44.25s/it]
NLL / N: 97.1851
10it [09:16, 55.70s/it]
preds: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 

1it [05:15, 315.18s/it]
NLL / N: 98.4897
2it [05:26, 223.97s/it]
NLL / N: 100.2908
3it [05:39, 160.76s/it]
NLL / N: 98.8324
4it [05:56, 117.48s/it]
NLL / N: 97.5683
5it [06:15, 87.98s/it] 
NLL / N: 97.2721
6it [06:37, 68.14s/it]
NLL / N: 97.2935
7it [07:02, 55.45s/it]
NLL / N: 97.3640
8it [07:27, 46.07s/it]
NLL / N: 97.4249
9it [07:51, 39.51s/it]
NLL / N: 97.4468
10it [08:15, 49.55s/it]
preds: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.,

3it [05:25, 154.28s/it]
NLL / N: 82.0231
4it [05:41, 112.67s/it]
NLL / N: 78.9629
5it [05:59, 84.44s/it] 
NLL / N: 78.3446
6it [06:20, 65.35s/it]
NLL / N: 78.1481
7it [06:45, 53.24s/it]
NLL / N: 78.1186
8it [07:11, 45.12s/it]
NLL / N: 79.2236
9it [07:34, 38.32s/it]
NLL / N: 95.0259
10it [07:57, 47.72s/it]
preds: tensor([1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1.,
        0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0.,
        0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0.,
        1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1.,
        1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 0., 1.

In [4]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_sin_K3_C5_1' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 3

/mnt/Cohortney/src/mds20_cohortney-main/src/DMHP/HP.py:26: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result = lambda w0: integrate.quad(upper_bound, w0, np.inf)
2.2222222222222223
D = 15
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 111.0257
1it [09:09, 549.30s/it]
NLL / N: 111.3133
2it [09:24, 389.06s/it]
NLL / N: 118.2010
3it [09:45, 278.53s/it]
NLL / N: 113.5088
4it [10:11, 202.93s/it]
NLL / N: 109.5573
5it [10:40, 150.74s/it]
NLL / N: 107.8193
6it [11:12, 115.01s/it]
NLL / N: 106.8610
7it [11:47, 91.07s/it] 
NLL / N: 106.2733
8it [12:21, 74.09s/it]
NLL / N: 105.8978
9it [12:56, 62.25s/it]
NLL / N: 105.6279
10it [13:31, 81.11s/it]
preds: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,

In [5]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_sin_K4_C5_1' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 4

/mnt/Cohortney/src/mds20_cohortney-main/src/DMHP/HP.py:26: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result = lambda w0: integrate.quad(upper_bound, w0, np.inf)
3.4984984984984986
D = 23
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 50.1980
1it [30:24, 1824.05s/it]
NLL / N: 31.7404
2it [32:21, 1311.95s/it]
NLL / N: 41.8991
3it [34:46, 962.11s/it] 
NLL / N: 45.7003
4it [37:47, 727.69s/it]
NLL / N: 45.5403
5it [41:18, 572.52s/it]
NLL / N: 44.3373
6it [45:14, 471.68s/it]
NLL / N: 42.4847
7it [49:35, 408.59s/it]
NLL / N: 40.6091
8it [53:54, 363.60s/it]
NLL / N: 40.0482
9it [58:11, 331.64s/it]
NLL / N: 41.1810
10it [1:02:27, 374.79s/it]
preds: tensor([1., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1

6it [46:54, 490.15s/it]
NLL / N: 39.1760
7it [51:45, 430.42s/it]
NLL / N: 37.8525
8it [56:34, 388.18s/it]
NLL / N: 36.8676
9it [1:01:22, 358.07s/it]
NLL / N: 37.9075
10it [1:06:13, 397.34s/it]
preds: tensor([3., 0., 0., 3., 0., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 0., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 0.,
        0., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        0., 3., 0., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,


In [6]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_sin_K5_C5_1' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 5

/mnt/Cohortney/src/mds20_cohortney-main/src/DMHP/HP.py:26: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result = lambda w0: integrate.quad(upper_bound, w0, np.inf)
2.8828828828828827
D = 19
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 101.3209
1it [19:57, 1197.10s/it]
NLL / N: 90.5178
2it [20:42, 851.48s/it] 
NLL / N: 88.2630
3it [21:39, 613.31s/it]
NLL / N: 87.7593
4it [22:49, 450.30s/it]
NLL / N: 89.8857
5it [24:11, 339.64s/it]
NLL / N: 92.5745
6it [25:41, 264.76s/it]
NLL / N: 95.5405
7it [27:21, 215.47s/it]
NLL / N: 97.0617
8it [29:03, 181.28s/it]
NLL / N: 96.9678
9it [30:42, 156.61s/it]
NLL / N: 96.7607
10it [32:21, 194.10s/it]
preds: tensor([1., 1., 1., 1., 3., 1., 2., 1., 3., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
        2., 1., 1., 1., 3., 1., 1., 1., 1., 1., 1., 2., 3., 1., 1., 1., 1., 1.,
        1., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1.,

0it [00:00, ?it/s]
NLL / N: 101.4446
1it [18:52, 1132.83s/it]
NLL / N: 92.9688
2it [19:40, 807.36s/it] 
NLL / N: 97.0237
3it [20:40, 583.14s/it]
NLL / N: 100.4489
4it [21:51, 429.55s/it]
NLL / N: 100.6880
5it [23:15, 325.71s/it]
NLL / N: 100.5514
6it [24:48, 256.02s/it]
NLL / N: 99.6288
7it [26:30, 209.72s/it]
NLL / N: 97.7954
8it [28:12, 177.44s/it]
NLL / N: 96.2344
9it [29:54, 154.84s/it]
NLL / N: 96.7708
10it [31:36, 189.67s/it]
preds: tensor([1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 2., 1., 1., 1.,
        4., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 2., 2., 1., 1., 1.,
        1., 1., 1., 2., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 2., 1., 1., 0.,
        1., 0., 2., 1., 1., 2., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 2.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 0., 1., 1., 1., 0., 2., 2., 1.,
        0., 1., 1., 1., 1., 1., 1., 2., 0., 1., 1., 1., 1., 1., 2., 2., 1., 1.,
        1., 0., 1., 2., 2., 1., 1., 2., 1., 1., 1., 1., 1., 0., 2., 1., 0., 1

In [7]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_trunc_K2_C5_1' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 2

0.5705705705705706
D = 4
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 106.8840
1it [00:57, 57.76s/it]
NLL / N: 105.6486
2it [01:06, 42.94s/it]
NLL / N: 107.4635
3it [01:16, 33.17s/it]
NLL / N: 105.9184
4it [01:29, 26.99s/it]
NLL / N: 99.6822
5it [01:45, 23.91s/it]
NLL / N: 97.6148
6it [02:02, 21.81s/it]
NLL / N: 97.0471
7it [02:22, 21.11s/it]
NLL / N: 96.9397
8it [02:40, 20.42s/it]
NLL / N: 96.9314
9it [02:59, 19.88s/it]
NLL / N: 96.9314
10it [03:19, 19.95s/it]
preds: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 

1it [00:55, 55.07s/it]
NLL / N: 103.4281
2it [01:03, 41.07s/it]
NLL / N: 107.5000
3it [01:18, 33.22s/it]
NLL / N: 105.0292
4it [01:31, 27.19s/it]
NLL / N: 99.2698
5it [01:46, 23.56s/it]
NLL / N: 97.4511
6it [02:05, 22.23s/it]
NLL / N: 96.9909
7it [02:25, 21.38s/it]
NLL / N: 96.9327
8it [02:43, 20.52s/it]
NLL / N: 96.9314
9it [03:06, 21.10s/it]
NLL / N: 96.9312
10it [03:25, 20.54s/it]
preds: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

4it [01:26, 26.16s/it]
NLL / N: 98.6145
5it [01:41, 22.91s/it]
NLL / N: 97.3196
6it [01:59, 21.25s/it]
NLL / N: 96.9707
7it [02:18, 20.72s/it]
NLL / N: 96.9319
8it [02:39, 20.72s/it]
NLL / N: 96.9312
9it [02:59, 20.69s/it]
NLL / N: 96.9315
10it [03:20, 20.02s/it]
preds: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0

In [8]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_trunc_K3_C5_1' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 3

0.960960960960961
D = 7
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 95.6635
1it [02:19, 139.34s/it]
NLL / N: 94.8957
2it [02:29, 100.61s/it]
NLL / N: 93.1641
3it [02:42, 74.36s/it] 
NLL / N: 89.9612
4it [02:59, 57.06s/it]
NLL / N: 88.2678
5it [03:18, 45.56s/it]
NLL / N: 87.7371
6it [03:38, 38.15s/it]
NLL / N: 87.4952
7it [04:05, 34.71s/it]
NLL / N: 87.4469
8it [04:29, 31.36s/it]
NLL / N: 87.4734
9it [04:51, 28.60s/it]
NLL / N: 87.5016
10it [05:13, 31.31s/it]
preds: tensor([1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
        1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1.,
        1., 2., 1., 1., 1., 0.

In [9]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_trunc_K4_C5_1' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 4

1.3963963963963963
D = 9
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 88.1115
1it [04:15, 255.46s/it]
NLL / N: 86.2546
2it [04:27, 182.46s/it]
NLL / N: 85.6049
3it [04:43, 132.41s/it]
NLL / N: 88.2709
4it [05:02, 98.53s/it] 
NLL / N: 91.5649
5it [05:25, 75.67s/it]
NLL / N: 94.1646
6it [05:50, 60.71s/it]
NLL / N: 101.5932
7it [06:21, 51.63s/it]
NLL / N: 105.9908
8it [06:48, 44.41s/it]
NLL / N: 106.1989
9it [07:16, 39.29s/it]
NLL / N: 107.0194
10it [07:44, 46.44s/it]
preds: tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 

In [10]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_trunc_K5_C5_1' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 5

1.7267267267267268
D = 11
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 96.8934
1it [06:46, 406.99s/it]
NLL / N: 99.8849
2it [07:05, 290.45s/it]
NLL / N: 105.4765
3it [07:27, 209.87s/it]
NLL / N: 104.9984
4it [07:54, 154.95s/it]
NLL / N: 103.7018
5it [08:23, 117.41s/it]
NLL / N: 103.9581
6it [08:57, 92.27s/it] 
NLL / N: 101.2307
7it [09:37, 76.50s/it]
NLL / N: 98.1346
8it [10:13, 64.48s/it]
NLL / N: 97.1495
9it [10:50, 56.13s/it]
NLL / N: 96.8377
10it [11:29, 68.94s/it]
preds: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 4., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 0., 1., 1., 1.,
        0., 1., 1., 

0it [00:00, ?it/s]
NLL / N: 97.0633
1it [06:48, 408.79s/it]
NLL / N: 99.3726
2it [07:05, 291.24s/it]
NLL / N: 103.4048
3it [07:26, 210.19s/it]
NLL / N: 102.2113
4it [07:54, 155.56s/it]
NLL / N: 100.8061
5it [08:25, 118.15s/it]
NLL / N: 99.7247
6it [09:00, 92.98s/it] 
NLL / N: 99.2308
7it [09:36, 76.09s/it]
NLL / N: 99.3979
8it [10:13, 64.32s/it]
NLL / N: 99.0718
9it [10:53, 57.08s/it]
NLL / N: 98.6104
10it [11:31, 69.15s/it]
preds: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    